In [1]:
import os
from dotenv import load_dotenv

# load .env file to environment
load_dotenv()

True

In [2]:
# Remaining imports
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain.chains import GraphCypherQAChain
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

from langchain.globals import set_debug
set_debug(False)

In [3]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")
llm_transformer = LLMGraphTransformer(llm=llm, node_properties=['name'])

In [4]:
text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person', properties={'name': 'Marie Curie'}), Node(id='Pierre Curie', type='Person', properties={'name': 'Pierre Curie'}), Node(id='University Of Paris', type='University', properties={'name': 'University of Paris'}), Node(id='Physicist', type='Field'), Node(id='Chemist', type='Field'), Node(id='Nobel Prize', type='Award'), Node(id='Radioactivity', type='Research'), Node(id='Curie Family', type='Family'), Node(id='1906', type='Year')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Physicist', type='Field'), type='CONDUCTED_RESEARCH'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Chemist', type='Field'), type='CONDUCTED_RESEARCH'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Nobel Prize', type='Award'), type='WON'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Nobel Prize', type='Award'), type='WON'), Relationship(sourc

In [5]:
# Insert into DB
graph = Neo4jGraph()
graph.add_graph_documents(graph_documents)
print("Done")

Done


In [ ]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'    
    return widget

w = showGraph()
display(w)

In [6]:
# Some few shot examples
examples = [
    {
        "question": "Who is a person married to?",
        "query": "MATCH (a:Person)-[:SPOUSE]->(:Person) RETURN count(DISTINCT a)",
    },
    {
        "question": "Who is a person living with?",
        "query": "MATCH (a:Person)-[:SPOUSE_OF]->(:Person) RETURN count(DISTINCT a)",
    }
]

example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)
prompt = FewShotPromptTemplate(
    examples=examples[:2],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. For entity primary keys, use the p id propoerty in queries. Given an input question, create a syntactically correct Cypher query to run.\n\nHere is the schema information\n{schema}.\n\nBelow are a number of examples of questions and their corresponding Cypher queries.",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question", "schema"],
)


# Query the graph database
chain = GraphCypherQAChain.from_llm(graph=graph, cypher_prompt=prompt, llm=llm, verbose=True)
response = chain.invoke({"query": "Who was Marie Curie married to?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person {id: "Marie Curie"})-[:SPOUSE]->(b:Person) RETURN b
Full Context:
[{'b': {'id': 'Pierre Curie'}}]

> Finished chain.


{'query': 'Who was Marie Curie married to?', 'result': 'Pierre Curie'}